# Configuracion del entorno

In [1]:
import sys
print(sys.executable)

/mnt/homeGPU/haoweihu/env/bin/python


In [1]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117


Looking in indexes: https://download.pytorch.org/whl/cu117
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch

print("Versión de torch:", torch.__version__)
print("CUDA disponible:", torch.cuda.is_available())
print("Dispositivos CUDA:", torch.cuda.device_count())
print("Nombre de la GPU:", torch.cuda.get_device_name(0) if torch.cuda.device_count() > 0 else "Ninguna GPU")


Versión de torch: 2.3.1
CUDA disponible: False
Dispositivos CUDA: 0
Nombre de la GPU: Ninguna GPU


In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="<0>"


In [4]:
# The first thing we should do is to apply hardware acceleration via GPU. So, we have to go to Editar - Configuracion del Cuaderno - Acelerador por Hardware GPU

import fastbook
fastbook.setup_book()
import fastai
import fastcore
import cv2

print(f'fastcore version {fastcore.__version__} installed')
print(f'fastai version {fastai.__version__} installed')
from nbdev.showdoc import *
from fastai.vision.all import *

fastcore version 1.7.27 installed
fastai version 2.7.19 installed


In [5]:
class LeNet(nn.Module):
  def __init__(self, num_classes=10):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
    self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.fc1 = nn.Linear(16 * 4 * 4, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.view(x.size(0), -1)  # Aplanar la salida
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [6]:
def umbralizacion_tri(img: PILImage):
    return img.quantize(colors=3)

def umbralizacion_bi(img: PILImage):
    return img.quantize(colors=2)

import cv2
def otsu_threshold_transform(img: PILImage):
    img_gray = img.convert("L")
    img_array = np.array(img_gray)
    _, thresholded = cv2.threshold(img_array, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return PILImage.create(thresholded)

In [7]:
from fastai.vision.all import *
from pathlib import Path
import time

def quantize3(train_path, name):

    # Crear el DataBlock para entrenamiento y validación
    dblock = DataBlock(
        blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
        get_items=get_image_files,
        splitter=GrandparentSplitter(train_name='training', valid_name='valid'),
        get_y=parent_label,
        item_tfms=[umbralizacion_tri]
    )

    dls = dblock.dataloaders(train_path, bs=64)

    # Obtener el número total de imágenes
    num_train = len(dls.train_ds)
    num_valid = len(dls.valid_ds)

    print(f"Número total de imágenes en entrenamiento: {num_train}")
    print(f"Número total de imágenes en validación: {num_valid}")

    # Obtener la distribución de clases
    class_counts = dls.train_ds.vocab  # Obtiene las clases
    print("\nClases:", class_counts)

    # Inicializar el modelo LeNet con 10 clases
    model = LeNet(num_classes=10)
    
    # Crear el objeto Learner
    learn = Learner(
        dls, 
        model, 
        loss_func=CrossEntropyLossFlat(), 
        metrics=[accuracy, Recall(average='macro'), F1Score(average='macro')], 
        cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=5)]
    )

    # Entrenamiento del modelo con fit_one_cycle
    start_time = time.time()
    learn.fit_one_cycle(30)
    end_time = time.time()
    print(f"\nTiempo de entrenamiento: {end_time - start_time:.2f} segundos")

    # Validación y reporte de métricas
    loss, acc, recall, f1 = learn.validate()
    print(f'Loss: {loss:.4f}')
    print(f'Accuracy: {acc:.4f}')
    print(f"Recall: {recall:.4f}")
    print(f'F1-Score: {f1:.4f}')

    # Exportar el modelo entrenado
    learn.export('/mnt/homeGPU/haoweihu/code/models/'+name+'.pkl')

    # Crear el DataBlock para los datos de prueba
    test_block = DataBlock(
        blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
        get_items=get_image_files,
        get_y=parent_label,
        splitter=IndexSplitter([]),
        item_tfms=[umbralizacion_tri]
    )

    test_dls = test_block.dataloaders("/mnt/homeGPU/haoweihu/code/dataset/original/fashion_mnist/test")

    # Cargar el modelo previamente exportado y remover el callback de EarlyStopping
    learn = load_learner('/mnt/homeGPU/haoweihu/code/models/'+name+'.pkl')
    learn.remove_cb(EarlyStoppingCallback)

    # Validar con el conjunto de test
    loss, acc, recall, f1 = learn.validate(dl=test_dls.train)
    print(f"Loss: {loss}")
    print(f"Accuracy: {acc}")
    print(f"Recall (macro): {recall}")
    print(f"F1 Score (macro): {f1}")
    


In [8]:
def quantize2(train_path, name):

    # Crear el DataBlock para entrenamiento y validación
    dblock = DataBlock(
        blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
        get_items=get_image_files,
        splitter=GrandparentSplitter(train_name='training', valid_name='valid'),
        get_y=parent_label,
        item_tfms=[umbralizacion_bi]
    )

    dls = dblock.dataloaders(train_path, bs=64)

    # Obtener el número total de imágenes
    num_train = len(dls.train_ds)
    num_valid = len(dls.valid_ds)

    print(f"Número total de imágenes en entrenamiento: {num_train}")
    print(f"Número total de imágenes en validación: {num_valid}")

    # Obtener la distribución de clases
    class_counts = dls.train_ds.vocab  # Obtiene las clases
    print("\nClases:", class_counts)

    # Inicializar el modelo LeNet con 10 clases
    model = LeNet(num_classes=10)
    
    # Crear el objeto Learner
    learn = Learner(
        dls, 
        model, 
        loss_func=CrossEntropyLossFlat(), 
        metrics=[accuracy, Recall(average='macro'), F1Score(average='macro')], 
        cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=5)]
    )

    # Entrenamiento del modelo con fit_one_cycle
    start_time = time.time()
    learn.fit_one_cycle(30)
    end_time = time.time()
    print(f"\nTiempo de entrenamiento: {end_time - start_time:.2f} segundos")

    # Validación y reporte de métricas
    loss, acc, recall, f1 = learn.validate()
    print(f'Loss: {loss:.4f}')
    print(f'Accuracy: {acc:.4f}')
    print(f"Recall: {recall:.4f}")
    print(f'F1-Score: {f1:.4f}')

    # Exportar el modelo entrenado
    learn.export('/mnt/homeGPU/haoweihu/code/models/'+name+'.pkl')

    # Crear el DataBlock para los datos de prueba
    test_block = DataBlock(
        blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
        get_items=get_image_files,
        get_y=parent_label,
        splitter=IndexSplitter([]),
        item_tfms=[umbralizacion_bi]
    )

    test_dls = test_block.dataloaders("/mnt/homeGPU/haoweihu/code/dataset/original/fashion_mnist/test")

    # Cargar el modelo previamente exportado y remover el callback de EarlyStopping
    learn = load_learner('/mnt/homeGPU/haoweihu/code/models/'+name+'.pkl')
    learn.remove_cb(EarlyStoppingCallback)

    # Validar con el conjunto de test
    loss, acc, recall, f1 = learn.validate(dl=test_dls.train)
    print(f"Loss: {loss}")
    print(f"Accuracy: {acc}")
    print(f"Recall (macro): {recall}")
    print(f"F1 Score (macro): {f1}")

In [9]:
def otsu(train_path, name):

    # Crear el DataBlock para entrenamiento y validación
    dblock = DataBlock(
        blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
        get_items=get_image_files,
        splitter=GrandparentSplitter(train_name='training', valid_name='valid'),
        get_y=parent_label,
        item_tfms=[otsu_threshold_transform]
    )

    dls = dblock.dataloaders(train_path, bs=64)

    # Obtener el número total de imágenes
    num_train = len(dls.train_ds)
    num_valid = len(dls.valid_ds)

    print(f"Número total de imágenes en entrenamiento: {num_train}")
    print(f"Número total de imágenes en validación: {num_valid}")

    # Obtener la distribución de clases
    class_counts = dls.train_ds.vocab  # Obtiene las clases
    print("\nClases:", class_counts)

    # Inicializar el modelo LeNet con 10 clases
    model = LeNet(num_classes=10)
    
    # Crear el objeto Learner
    learn = Learner(
        dls, 
        model, 
        loss_func=CrossEntropyLossFlat(), 
        metrics=[accuracy, Recall(average='macro'), F1Score(average='macro')], 
        cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=5)]
    )

    # Entrenamiento del modelo con fit_one_cycle
    start_time = time.time()
    learn.fit_one_cycle(30)
    end_time = time.time()
    print(f"\nTiempo de entrenamiento: {end_time - start_time:.2f} segundos")

    # Validación y reporte de métricas
    loss, acc, recall, f1 = learn.validate()
    print(f'Loss: {loss:.4f}')
    print(f'Accuracy: {acc:.4f}')
    print(f"Recall: {recall:.4f}")
    print(f'F1-Score: {f1:.4f}')

    # Exportar el modelo entrenado
    learn.export('/mnt/homeGPU/haoweihu/code/models/'+name+'.pkl')

    # Crear el DataBlock para los datos de prueba
    test_block = DataBlock(
        blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
        get_items=get_image_files,
        get_y=parent_label,
        splitter=IndexSplitter([]),
        item_tfms=[otsu_threshold_transform]
    )

    test_dls = test_block.dataloaders("/mnt/homeGPU/haoweihu/code/dataset/original/fashion_mnist/test")

    # Cargar el modelo previamente exportado y remover el callback de EarlyStopping
    learn = load_learner('/mnt/homeGPU/haoweihu/code/models/'+name+'.pkl')
    learn.remove_cb(EarlyStoppingCallback)

    # Validar con el conjunto de test
    loss, acc, recall, f1 = learn.validate(dl=test_dls.train)
    print(f"Loss: {loss}")
    print(f"Accuracy: {acc}")
    print(f"Recall (macro): {recall}")
    print(f"F1 Score (macro): {f1}")

In [7]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/fmnist/example1", "fmnist_dist_quant3_ex1")


Número total de imágenes en entrenamiento: 9558
Número total de imágenes en validación: 10800

Clases: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'Trouser']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.299190,2.293546,0.111111,0.111111,0.022222,00:36
1,2.237795,2.208439,0.111111,0.111111,0.022222,00:36
2,2.205384,2.199956,0.111111,0.111111,0.022222,00:37
3,2.202606,2.202552,0.111111,0.111111,0.022222,00:36
4,2.201644,2.199945,0.111111,0.111111,0.022222,00:36
5,2.201344,2.200041,0.111111,0.111111,0.022222,00:37
6,2.192366,2.180304,0.174167,0.174167,0.095831,00:36
7,1.168388,0.939325,0.589630,0.589630,0.557797,00:36
8,0.879290,0.869906,0.617593,0.617593,0.571751,00:36
9,0.796932,0.799214,0.673704,0.673704,0.660758,00:37



Tiempo de entrenamiento: 1118.53 segundos


Loss: 0.5997
Accuracy: 0.7540
Recall: 0.7540
F1-Score: 0.7495


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 5.471940517425537
Accuracy: 0.6722756624221802
Recall (macro): 0.6723202012411545
F1 Score (macro): 0.6492094738835985


In [8]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/fmnist/example2", "fmnist_dist_quant3_ex2")    

Número total de imágenes en entrenamiento: 9558
Número total de imágenes en validación: 10800

Clases: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'Trouser']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.293203,2.289094,0.111111,0.111111,0.022222,00:52
1,2.248045,2.215159,0.218426,0.218426,0.080296,00:37
2,2.204494,2.202621,0.111111,0.111111,0.022222,00:36
3,2.201647,2.198843,0.111111,0.111111,0.022222,00:36
4,2.199493,2.197696,0.111111,0.111111,0.022222,00:36
5,2.138229,1.999221,0.270463,0.270463,0.173959,00:36
6,1.395617,1.257854,0.450833,0.450833,0.379119,00:36
7,1.162276,1.090506,0.539167,0.539167,0.497352,00:37
8,0.982180,0.933464,0.653148,0.653148,0.624427,00:36
9,0.865166,0.854700,0.663796,0.663796,0.640900,00:36



Tiempo de entrenamiento: 1131.27 segundos


Loss: 0.5920
Accuracy: 0.7614
Recall: 0.7614
F1-Score: 0.7576


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 3.3324060440063477
Accuracy: 0.6785857081413269
Recall (macro): 0.6785485967342152
F1 Score (macro): 0.6543375249617357


In [9]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/fmnist/example3", "fmnist_dist_quant3_ex3")    

Número total de imágenes en entrenamiento: 9558
Número total de imágenes en validación: 10800

Clases: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'Trouser']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.291121,2.278909,0.111111,0.111111,0.022222,00:40
1,2.220753,2.202559,0.125093,0.125093,0.042805,00:40
2,2.203164,2.201597,0.111111,0.111111,0.022612,00:37
3,2.201788,2.201781,0.111111,0.111111,0.022222,00:36
4,2.200622,2.199724,0.111111,0.111111,0.022222,00:36
5,2.156379,2.030285,0.300000,0.300000,0.175913,00:36
6,1.296635,1.136606,0.530926,0.530926,0.483730,00:36
7,0.981475,0.907306,0.638241,0.638241,0.626521,00:36
8,0.848731,0.805577,0.677778,0.677778,0.661587,00:36
9,0.762341,0.773884,0.677870,0.677870,0.658311,00:36



Tiempo de entrenamiento: 1182.25 segundos


Loss: 0.5767
Accuracy: 0.7728
Recall: 0.7728
F1-Score: 0.7698


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 3.6212189197540283
Accuracy: 0.6886017918586731
Recall (macro): 0.6886334535117888
F1 Score (macro): 0.6656681332128941


In [12]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/fmnist/example1", "fmnist_dist_quant2_ex1")


Número total de imágenes en entrenamiento: 9558
Número total de imágenes en validación: 10800

Clases: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'Trouser']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.292473,2.285667,0.111111,0.111111,0.022222,00:36
1,2.233238,2.204773,0.111111,0.111111,0.022222,00:36
2,2.202892,2.201037,0.111111,0.111111,0.022222,00:36
3,2.200992,2.199756,0.111111,0.111111,0.022222,00:37
4,2.201593,2.199844,0.111111,0.111111,0.022222,00:37
5,2.191821,2.175936,0.221574,0.221574,0.081156,00:36
6,1.448101,1.132736,0.555370,0.555370,0.488632,00:36
7,0.893550,0.824334,0.690093,0.690093,0.681719,00:36
8,0.759897,0.757657,0.686667,0.686667,0.672523,00:37
9,0.706986,0.744572,0.715463,0.715463,0.707074,00:37



Tiempo de entrenamiento: 1116.22 segundos


Loss: 0.5746
Accuracy: 0.7794
Recall: 0.7794
F1-Score: 0.7773


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 3.2297489643096924
Accuracy: 0.6953125
Recall (macro): 0.6953160192686964
F1 Score (macro): 0.6718404911744955


In [13]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/fmnist/example2", "fmnist_dist_quant2_ex2")

Número total de imágenes en entrenamiento: 9558
Número total de imágenes en validación: 10800

Clases: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'Trouser']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.297233,2.292733,0.111111,0.111111,0.022222,00:36
1,2.245719,2.214743,0.111111,0.111111,0.022222,00:38
2,2.203953,2.200561,0.111111,0.111111,0.022222,00:36
3,2.202197,2.198940,0.111111,0.111111,0.022222,00:36
4,2.201444,2.200091,0.111111,0.111111,0.022222,00:36
5,2.199990,2.199371,0.111111,0.111111,0.022222,00:38
6,2.194697,2.181227,0.111574,0.111574,0.026270,00:37
7,1.277807,1.079598,0.571574,0.571574,0.536137,00:37
8,0.911273,0.891881,0.639815,0.639815,0.596888,00:36
9,0.822222,0.841918,0.638796,0.638796,0.591266,00:50



Tiempo de entrenamiento: 1175.60 segundos


Loss: 0.6623
Accuracy: 0.7280
Recall: 0.7280
F1-Score: 0.7239


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 6.216012954711914
Accuracy: 0.6520432829856873
Recall (macro): 0.6519242441692713
F1 Score (macro): 0.6288844896087415


In [14]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/fmnist/example3", "fmnist_dist_quant2_ex3")
    

Número total de imágenes en entrenamiento: 9558
Número total de imágenes en validación: 10800

Clases: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'Trouser']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.287766,2.281071,0.111111,0.111111,0.022222,00:36
1,2.227087,2.205521,0.111111,0.111111,0.022222,00:36
2,2.204284,2.199811,0.111111,0.111111,0.022222,00:37
3,2.202828,2.200515,0.111111,0.111111,0.022222,00:36
4,2.202712,2.199817,0.111111,0.111111,0.022222,00:35
5,2.199592,2.195078,0.111111,0.111111,0.022222,00:36
6,1.515736,1.177281,0.517222,0.517222,0.469896,00:36
7,0.953931,0.875066,0.643241,0.643241,0.602019,00:36
8,0.817731,0.795532,0.661759,0.661759,0.629284,00:37
9,0.762184,0.766459,0.659074,0.659074,0.613263,00:37



Tiempo de entrenamiento: 1113.93 segundos


Loss: 0.6122
Accuracy: 0.7545
Recall: 0.7545
F1-Score: 0.7503


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 5.593207836151123
Accuracy: 0.6724759340286255
Recall (macro): 0.6725505432381833
F1 Score (macro): 0.6496822438838671


In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/fmnist/example1", "fmnist_dist_otsu_ex1")


Could not do one pass in your dataloader, there is something wrong in it. Please see the stack trace below:


NameError: name 'cv2' is not defined

In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/fmnist/example2", "fmnist_dist_otsu_ex2")

In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/fmnist/example3", "fmnist_dist_otsu_ex3")

# Distilled 005

In [16]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/fmnist/example1", "fmnist_dist5_quant3_ex1")    

Número total de imágenes en entrenamiento: 8820
Número total de imágenes en validación: 10800

Clases: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'Trouser']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.289059,2.283030,0.111111,0.111111,0.022222,00:53
1,2.240986,2.212287,0.111111,0.111111,0.022222,00:35
2,2.204696,2.199702,0.111111,0.111111,0.022222,00:35
3,2.201362,2.201880,0.111111,0.111111,0.022222,00:35
4,2.201580,2.200454,0.111111,0.111111,0.022222,00:36
5,2.200686,2.202853,0.111111,0.111111,0.022222,00:35
6,2.200670,2.198260,0.111111,0.111111,0.022222,00:35
7,2.199835,2.199364,0.111111,0.111111,0.022222,00:35
8,2.193053,2.180633,0.252315,0.252315,0.155627,00:35
9,1.499562,1.251713,0.443611,0.443611,0.368150,00:35


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe9d0dc13a0>
Traceback (most recent call last):
  File "/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/mnt/homeGPU/haoweihu/env/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe9d0dc13a0>
Traceback (most recent call last):
  File "/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/torch/utils/data/dataloader


Tiempo de entrenamiento: 1091.31 segundos


Loss: 0.6458
Accuracy: 0.7364
Recall: 0.7364
F1-Score: 0.7317


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 5.975671291351318
Accuracy: 0.6590544581413269
Recall (macro): 0.6588764683025735
F1 Score (macro): 0.6313326581286031


In [17]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/fmnist/example2", "fmnist_dist5_quant3_ex2")    

Número total de imágenes en entrenamiento: 8820
Número total de imágenes en validación: 10800

Clases: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'Trouser']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.305458,2.297357,0.111111,0.111111,0.022222,00:56
1,2.235032,2.207505,0.111111,0.111111,0.022222,00:36
2,2.205864,2.203711,0.111111,0.111111,0.022222,00:35
3,2.203284,2.201398,0.111111,0.111111,0.022222,00:35
4,2.202165,2.198762,0.111111,0.111111,0.022222,00:35
5,2.180627,2.127552,0.152130,0.152130,0.065876,00:34
6,1.143271,0.860668,0.660648,0.660648,0.655030,00:34
7,0.795071,0.766656,0.691204,0.691204,0.670490,00:34
8,0.737284,0.735895,0.679444,0.679444,0.661630,00:35
9,0.714979,0.706071,0.708704,0.708704,0.686345,00:35



Tiempo de entrenamiento: 1092.36 segundos


Loss: 0.5869
Accuracy: 0.7673
Recall: 0.7673
F1-Score: 0.7645


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 3.1303372383117676
Accuracy: 0.6832932829856873
Recall (macro): 0.6832135118555308
F1 Score (macro): 0.6598731493178043


In [18]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/fmnist/example3", "fmnist_dist5_quant3_ex3")    

Número total de imágenes en entrenamiento: 8820
Número total de imágenes en validación: 10800

Clases: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'Trouser']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.300218,2.294435,0.111111,0.111111,0.022222,00:39
1,2.248729,2.215415,0.111111,0.111111,0.022222,00:38
2,2.204527,2.204623,0.111111,0.111111,0.022222,00:35
3,2.202845,2.201015,0.111111,0.111111,0.022222,00:36
4,2.201793,2.200197,0.111111,0.111111,0.022222,00:35
5,2.201980,2.198687,0.111111,0.111111,0.022222,00:35
6,2.199677,2.200969,0.111111,0.111111,0.022222,00:35
7,2.167569,2.021162,0.424167,0.424167,0.314825,00:35
8,1.042524,0.838048,0.667407,0.667407,0.658138,00:35
9,0.807211,0.753296,0.684815,0.684815,0.666991,00:35



Tiempo de entrenamiento: 1080.70 segundos


Loss: 0.5901
Accuracy: 0.7610
Recall: 0.7610
F1-Score: 0.7576


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 4.776061534881592
Accuracy: 0.6806890964508057
Recall (macro): 0.6805746366027897
F1 Score (macro): 0.6572127142102921


In [19]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/fmnist/example1", "fmnist_dist5_quant2_ex1")    

Número total de imágenes en entrenamiento: 8820
Número total de imágenes en validación: 10800

Clases: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'Trouser']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.287992,2.281643,0.111111,0.111111,0.022222,00:36
1,2.232498,2.206647,0.111111,0.111111,0.022222,00:35
2,2.204902,2.199692,0.111111,0.111111,0.022222,00:36
3,2.203858,2.200699,0.111111,0.111111,0.022222,00:35
4,2.202477,2.200274,0.111111,0.111111,0.022222,00:35
5,2.200476,2.196090,0.111111,0.111111,0.022222,00:35
6,1.845989,1.194698,0.547593,0.547593,0.480904,00:35
7,0.986471,0.868917,0.630000,0.630000,0.602648,00:35
8,0.834697,0.800464,0.680926,0.680926,0.643664,00:36
9,0.772875,0.752927,0.697315,0.697315,0.687202,00:34



Tiempo de entrenamiento: 1076.48 segundos


Loss: 0.5845
Accuracy: 0.7706
Recall: 0.7706
F1-Score: 0.7677


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 4.021027565002441
Accuracy: 0.6862980723381042
Recall (macro): 0.6862823381547025
F1 Score (macro): 0.6625255594430106


In [20]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/fmnist/example2", "fmnist_dist5_quant2_ex2")    

Número total de imágenes en entrenamiento: 8820
Número total de imágenes en validación: 10800

Clases: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'Trouser']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.300745,2.297102,0.111111,0.111111,0.022222,00:35
1,2.256532,2.219596,0.111111,0.111111,0.022222,00:37
2,2.206268,2.200183,0.111111,0.111111,0.022222,00:34
3,2.201842,2.199474,0.111111,0.111111,0.022222,00:35
4,2.201610,2.199322,0.111111,0.111111,0.022222,00:35
5,2.200503,2.197863,0.111111,0.111111,0.022222,00:34
6,2.196259,2.191545,0.111111,0.111111,0.022222,00:34
7,1.946970,1.597234,0.307130,0.307130,0.185253,00:35
8,1.362675,1.229223,0.450648,0.450648,0.379333,00:35
9,1.099442,1.013620,0.647963,0.647963,0.614423,00:35



Tiempo de entrenamiento: 1066.24 segundos


Loss: 0.6707
Accuracy: 0.7350
Recall: 0.7350
F1-Score: 0.7251


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 3.781470775604248
Accuracy: 0.6568509340286255
Recall (macro): 0.6569122249041836
F1 Score (macro): 0.625614895601404


In [21]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/fmnist/example3", "fmnist_dist5_quant2_ex3")    

Número total de imágenes en entrenamiento: 8820
Número total de imágenes en validación: 10800

Clases: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'Trouser']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.284939,2.279579,0.111111,0.111111,0.022222,00:58
1,2.235099,2.208606,0.111111,0.111111,0.022222,00:35
2,2.203775,2.199355,0.111111,0.111111,0.022222,00:34
3,2.201698,2.199430,0.111111,0.111111,0.022222,00:34
4,2.200683,2.201525,0.133148,0.133148,0.043056,00:34
5,2.168657,2.082853,0.222870,0.222870,0.085653,00:34
6,1.165439,0.966191,0.607870,0.607870,0.577894,00:34
7,0.918246,0.884179,0.622870,0.622870,0.614859,00:34
8,0.834566,0.798990,0.694537,0.694537,0.680580,00:34
9,0.780912,0.735831,0.713426,0.713426,0.707315,00:35



Tiempo de entrenamiento: 1084.76 segundos


Loss: 0.6085
Accuracy: 0.7656
Recall: 0.7656
F1-Score: 0.7626


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 3.763075590133667
Accuracy: 0.6857972741127014
Recall (macro): 0.685959759351075
F1 Score (macro): 0.6609049357168979


In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/fmnist/example1", "fmnist_dist5_otsu_ex1")    

In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/fmnist/example2", "fmnist_dist5_otsu_ex2")    

In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/fmnist/example3", "fmnist_dist5_otsu_ex3")    

# Distilled 0001

In [22]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/fmnist/example1", "fmnist_dist01_quant3_ex1")    

Número total de imágenes en entrenamiento: 20160
Número total de imágenes en validación: 10800

Clases: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'Trouser']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.282602,2.268945,0.111111,0.111111,0.022222,01:34
1,2.201866,2.199936,0.111111,0.111111,0.022222,00:57
2,2.201321,2.199766,0.111111,0.111111,0.022222,00:57
3,2.191087,2.187652,0.172593,0.172593,0.100912,00:57
4,1.178968,1.312205,0.461019,0.461019,0.439842,00:56
5,0.872710,0.965508,0.629444,0.629444,0.606159,00:56
6,0.742371,0.980582,0.628333,0.628333,0.625036,00:56
7,0.672302,0.864878,0.632870,0.632870,0.620744,00:56
8,0.594130,0.743056,0.690000,0.690000,0.676994,00:58
9,0.539765,0.709990,0.709352,0.709352,0.699348,00:57



Tiempo de entrenamiento: 1762.52 segundos


Loss: 0.5231
Accuracy: 0.8122
Recall: 0.8122
F1-Score: 0.8121


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 2.9230995178222656
Accuracy: 0.7198517918586731
Recall (macro): 0.7197825718712322
F1 Score (macro): 0.6966084282227559


In [23]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/fmnist/example2", "fmnist_dist01_quant3_ex2")    

Número total de imágenes en entrenamiento: 20160
Número total de imágenes en validación: 10800

Clases: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'Trouser']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.280638,2.265463,0.111111,0.111111,0.022222,01:17
1,2.201409,2.202510,0.111111,0.111111,0.022222,00:56
2,2.199523,2.199269,0.111111,0.111111,0.022222,00:57
3,2.120353,2.002099,0.256944,0.256944,0.147927,00:58
4,0.882941,0.967544,0.630833,0.630833,0.606619,00:59
5,0.758838,0.853458,0.670556,0.670556,0.651470,00:58
6,0.632843,0.832109,0.654167,0.654167,0.636785,00:59
7,0.583387,0.716269,0.715370,0.715370,0.711424,00:58
8,0.551262,0.653680,0.740278,0.740278,0.735995,00:58
9,0.510839,0.633776,0.753056,0.753056,0.748914,00:57



Tiempo de entrenamiento: 1756.08 segundos


Loss: 0.5464
Accuracy: 0.7954
Recall: 0.7954
F1-Score: 0.7954


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 2.7298073768615723
Accuracy: 0.7086338400840759
Recall (macro): 0.7084287154322964
F1 Score (macro): 0.6864153207063877


In [ ]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/fmnist/example3", "fmnist_dist01_quant3_ex3")    

Número total de imágenes en entrenamiento: 20160
Número total de imágenes en validación: 10800

Clases: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'Trouser']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time


In [ ]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/fmnist/example1", "fmnist_dist01_quant2_ex1")    

In [ ]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/fmnist/example2", "fmnist_dist01_quant2_ex2")    

In [ ]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/fmnist/example3", "fmnist_dist01_quant2_ex3")    

In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/fmnist/example1", "fmnist_dist01_otsu_ex1")    

In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/fmnist/example2", "fmnist_dist01_otsu_ex2")    

In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/fmnist/example3", "fmnist_dist01_otsu_ex3")    